In [16]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap
from my_tools import file_dic, plot_dic
import matplotlib.ticker as plticker
from mpl_toolkits.basemap import Basemap, addcyclic
import string
from matplotlib.patches import Polygon
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings("ignore")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
path = '/media/onno/Volume/'

In [5]:
file_env = 'era51_mars_env_wledit2000-10000_latavg_v300_79-19_6hourly_smoothed.nc'
file_cp = 'era51_mars_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_setvrange_-100to100.nc'
file_t850 = 'era51_mars_t850_79-19_6hourly_anom_from_smoothed04_clim.nc'
file_v300 = 'era51_mars_v300_wledit2000-10000_latavg_79-19_6hourly.nc'
file_z500 = 'era51_mars_phi500_79-19_6hourly.nc'
file_mslp = 'era5_mslp_79-19_6hourly_remapbil2x2.nc'
file_prec = 'era5_totprecip_79-19_1hourly_remapbil2x2_daysum.nc'
file_z500_anom = 'era51_mars_phi500_79-19_6hourly_anom_from_smoothed04_clim_daymean.nc'

era_env = xr.open_dataset(path+file_env)
era_cp = xr.open_dataset(path+file_cp)
era_v300 = xr.open_dataset(path+file_v300)
era_z500 = xr.open_dataset(path+file_z500)
era_mslp = xr.open_dataset(path+file_mslp)
era_t850 = xr.open_dataset(path+file_t850)
era_prec = xr.open_dataset(path+file_prec)
era_z500_anom = xr.open_dataset(path+file_z500_anom)
era_prec['time'] = [era_prec['time'].values[i] - pd.Timedelta(15,'h') if i==0 else 
                   era_prec['time'].values[i] - pd.Timedelta(11.5,'h')
                    for i in range(len( era_prec['time'].values))]
era_prec = era_prec.drop('time_bnds')
era_z500_anom['time'] = [era_z500_anom['time'].values[i] - pd.Timedelta(9,'h')
                    for i in range(len(era_z500_anom['time'].values))]
era_z500_anom = era_z500_anom.drop('time_bnds')


In [43]:

eraz = {
#         'v300':era_v300,
#        'z500':era_z500,
#        'mslp':era_mslp,
#        't850':era_t850,
       'prec':era_prec,
#        'env':era_env,
#        'cp':era_cp,
#         'z500_anom':era_z500_anom
}

seasonz = ['DJF','MAM','JJA','SON']
modelz = ['GFS','ERA5RF']
eventz = ['persistent_hw','persistent_cw']

countryz={
'Finland':{'good':'/GFS_T850/rank_forecast/good_forecasts_{}_lon_22_30_lat_68_60_{}_{}_lead_day_5.txt',
          'bad':'/GFS_T850/rank_forecast/bad_forecasts_{}_lon_22_30_lat_68_60_{}_{}_lead_day_5.txt'},
'Germany':{'good':'/GFS_T850/rank_forecast/good_forecasts_{}_lon_8_16_lat_54_46_{}_{}_lead_day_5.txt',
          'bad':'/GFS_T850/rank_forecast/bad_forecasts_{}_lon_8_16_lat_54_46_{}_{}_lead_day_5.txt'},
'Spain':{'good':'/GFS_T850/rank_forecast/good_forecasts_{}_lon_352_360_lat_44_36_{}_{}_lead_day_5.txt',
          'bad':'/GFS_T850/rank_forecast/bad_forecasts_{}_lon_352_360_lat_44_36_{}_{}_lead_day_5.txt'},
}








for country in countryz:
    for model in modelz:
        for event in eventz:
            for season in seasonz:
                for key in eraz:
                    if 'lat' in list(eraz[key].dims):
                        eraz[key]=eraz[key].rename({'lat':'latitude','lon':'longitude'})
                    file_bad = countryz[country]['bad']
                    file_good = countryz[country]['good']
                    bad_fcst = pd.read_csv(path+file_bad.format(event,season,model),index_col=0)
                    good_fcst = pd.read_csv(path+file_good.format(event,season,model),index_col=0)
                    bad_compositez = []
                    good_compositez = []
                    for date in bad_fcst.index:
                        date = pd.Timestamp(date)
                        begin_date = date - pd.Timedelta(7*24,'hours')
                        end_date = date + pd.Timedelta(5*24,'hours')
                        date_range = pd.date_range(begin_date,end_date,freq="24h")
                        era_select = eraz[key].sel(time=date_range)
                        era_select['time'] = np.arange(-7,6)
                        bad_compositez.append(era_select)
                    bad_composite_all = xr.concat(bad_compositez,'time').squeeze()
                    bad_composite = bad_composite_all.groupby('time').mean()
                    bad_composite_all_4D = my_tools.dataset_3D_to_4D(bad_composite_all)
                    bad_composite.to_netcdf(path+'/fcst_composite/reanalysis/composite_{}_bad_forecasts_{}_{}_{}_{}.nc'.format(key,event,season,country,model))
                    for date in good_fcst.index:
                        date = pd.Timestamp(date)
                        begin_date = date - pd.Timedelta(7*24,'hours')
                        end_date = date + pd.Timedelta(5*24,'hours')
                        date_range = pd.date_range(begin_date,end_date,freq="24h")
                        era_select = eraz[key].sel(time=date_range)
                        era_select['time'] = np.arange(-7,6)
                        good_compositez.append(era_select)
                    good_composite_all = xr.concat(good_compositez,'time').squeeze()
                    good_composite = good_composite_all.groupby('time').mean()
                    good_composite_all_4D = my_tools.dataset_3D_to_4D(good_composite_all)
                    good_composite.to_netcdf(path+'/fcst_composite/reanalysis/composite_{}_good_forecasts_{}_{}_{}_{}.nc'.format(key,event,season,country,model))

                    var = list(good_composite.keys())[0]
                    p_diff = ttest_ind(good_composite_all_4D[var].values,bad_composite_all_4D[var].values,
                                      axis=0,equal_var=False,nan_policy='omit')[1]
                    p_diff_ds = good_composite_all_4D.sel(N=1).rename({var:'p'})
                    p_diff_ds['p'].values = p_diff
                    p_diff_ds.to_netcdf(path+'/fcst_composite/reanalysis/composite_{}_diff_p_value_{}_{}_{}_{}.nc'.format(key,event,season,country,model))                

    for model in modelz:
        for event in eventz:
            for key in eraz:
                if 'lat' in list(eraz[key].dims):
                    eraz[key]=eraz[key].rename({'lat':'latitude','lon':'longitude'})
                file_bad = countryz[country]['bad']
                file_good = countryz[country]['good']
                for i,season in enumerate(seasonz):
                    if i == 0:
                        bad_fcst = pd.read_csv(path+file_bad.format(event,season,model),index_col=0)
                        good_fcst = pd.read_csv(path+file_good.format(event,season,model),index_col=0)
                    else:
                        bad_fcst_new = pd.read_csv(path+file_bad.format(event,season,model),index_col=0)
                        good_fcst_new = pd.read_csv(path+file_good.format(event,season,model),index_col=0)
                        bad_fcst=bad_fcst.append(bad_fcst_new)
                        good_fcst=good_fcst.append(good_fcst_new)
                bad_compositez = []
                good_compositez = []
                for date in bad_fcst.index:
                    date = pd.Timestamp(date)
                    begin_date = date - pd.Timedelta(7*24,'hours')
                    end_date = date + pd.Timedelta(5*24,'hours')
                    date_range = pd.date_range(begin_date,end_date,freq="24h")
                    era_select = eraz[key].sel(time=date_range)
                    era_select['time'] = np.arange(-7,6)
                    bad_compositez.append(era_select)
                bad_composite_all = xr.concat(bad_compositez,'time').squeeze()
                bad_composite = bad_composite_all.groupby('time').mean()
                bad_composite_all_4D = my_tools.dataset_3D_to_4D(bad_composite_all)
                bad_composite.to_netcdf(path+'/fcst_composite/reanalysis/composite_{}_bad_forecasts_{}_annual_{}_{}.nc'.format(key,event,country,model))
                for date in good_fcst.index:
                    date = pd.Timestamp(date)
                    begin_date = date - pd.Timedelta(7*24,'hours')
                    end_date = date + pd.Timedelta(5*24,'hours')
                    date_range = pd.date_range(begin_date,end_date,freq="24h")
                    era_select = eraz[key].sel(time=date_range)
                    era_select['time'] = np.arange(-7,6)
                    good_compositez.append(era_select)
                good_composite_all = xr.concat(good_compositez,'time').squeeze()
                good_composite = good_composite_all.groupby('time').mean()
                good_composite_all_4D = my_tools.dataset_3D_to_4D(good_composite_all)
                good_composite.to_netcdf(path+'/fcst_composite/reanalysis/composite_{}_good_forecasts_{}_annual_{}_{}.nc'.format(key,event,country,model))
                var = list(good_composite.keys())[0]
                p_diff = ttest_ind(good_composite_all_4D[var].values,bad_composite_all_4D[var].values,
                                  axis=0,equal_var=False,nan_policy='omit')[1]
                p_diff_ds = good_composite_all_4D.sel(N=1).rename({var:'p'})
                p_diff_ds['p'].values = p_diff
                p_diff_ds.to_netcdf(path+'/fcst_composite/reanalysis/composite_{}_diff_p_value_{}_annual_{}_{}.nc'.format(key,event,country,model))          




composite MSLP and Z500

In [21]:
plt.rcParams['hatch.color'] = 'w'
seasonz = ['DJF','MAM','JJA','SON','annual']
modelz = ['GFS','ERA5RF']
eventz = ['persistent_hw','persistent_cw']
event_labelz = ['Warm Extreme','Cold Extreme']
regionz = ['Germany','Finland','Spain']
file_good_mslp = '/fcst_composite/reanalysis/composite_mslp_good_forecasts_{}_{}_{}_{}.nc'
file_bad_mslp = '/fcst_composite/reanalysis/composite_mslp_bad_forecasts_{}_{}_{}_{}.nc'
file_p_mslp = '/fcst_composite/reanalysis/composite_mslp_diff_p_value_{}_{}_{}_{}.nc'
file_good_z500 = '/fcst_composite/reanalysis/composite_z500_good_forecasts_{}_{}_{}_{}.nc'
file_bad_z500 = '/fcst_composite/reanalysis/composite_z500_bad_forecasts_{}_{}_{}_{}.nc'
file_p_z500 = '/fcst_composite/reanalysis/composite_z500_diff_p_value_{}_{}_{}_{}.nc'

for region in regionz:
    for model in modelz:
        for j, event in enumerate(eventz):
            for season in seasonz:
                fig,axz = plt.subplots(6,2,figsize=(18,12))
                ds_good_mslp = xr.open_dataset(path+file_good_mslp.format(event,season,region,model))
                ds_bad_mslp = xr.open_dataset(path+file_bad_mslp.format(event,season,region,model))
                ds_good_z500 = xr.open_dataset(path+file_good_z500.format(event,season,region,model))
                ds_bad_z500 = xr.open_dataset(path+file_bad_z500.format(event,season,region,model))      
                ds_p_mslp = xr.open_dataset(path+file_p_mslp.format(event,season,region,model))
                ds_p_z500 = xr.open_dataset(path+file_p_z500.format(event,season,region,model))
                days_to_plot = np.arange(-6,5,2)
                for i in range(len(days_to_plot)):
                    ds_p_z500_day = ds_p_z500.sel(days=days_to_plot[i])
                    ds_p_z500_day_bool = ds_p_z500_day.p<0.05
                    ds_p_z500_day_cyclic, lon_cyclic = addcyclic(ds_p_z500_day_bool.values, ds_p_z500_day.longitude.values)
                    ds_good_mslp_day = ds_good_mslp.sel(time=days_to_plot[i])
                    ds_good_mslp_day_cyclic, lon_cyclic = addcyclic(ds_good_mslp_day.var151.values, ds_good_mslp_day.longitude.values)
                    ds_good_z500_day = ds_good_z500.sel(time=days_to_plot[i])
                    ds_good_z500_day_cyclic, lon_cyclic = addcyclic(ds_good_z500_day.z.values, ds_good_z500_day.longitude.values)                
                    lon,lat = np.meshgrid(lon_cyclic,ds_good_mslp_day.latitude.values)

                    ax_good = axz[i,0]
                    ax_good.set_title('Good Forecasts Day {}'.format(days_to_plot[i]))
                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_good,
                                llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    m.contour(lon,lat,ds_good_mslp_day_cyclic/100,latlon='true',
                              vmin=970,vmax=1040,levels=np.arange(970,1041,5),cmap='Greys')
                    m.contourf(lon,lat,ds_good_z500_day_cyclic/100,latlon='true',
                              vmin=450,vmax=600,cmap=ncl_colormap(),levels=np.linspace(450,600,30),extend='both')
                    m.contourf(lon,lat,ds_p_z500_day_cyclic,3,latlon='true',
                              colors='none',hatches=[None,'xx'])

                    ds_bad_mslp_day = ds_bad_mslp.sel(time=days_to_plot[i])
                    ds_bad_mslp_day_cyclic, lon_cyclic = addcyclic(ds_bad_mslp_day.var151.values, ds_bad_mslp_day.longitude.values)
                    ds_bad_z500_day = ds_bad_z500.sel(time=days_to_plot[i])
                    ds_bad_z500_day_cyclic, lon_cyclic = addcyclic(ds_bad_z500_day.z.values, ds_bad_z500_day.longitude.values)
                    lon,lat = np.meshgrid(lon_cyclic,ds_bad_mslp_day.latitude.values)

                    ax_bad = axz[i,1]
                    ax_bad.set_title('Bad Forecasts Day {}'.format(days_to_plot[i]))

                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_bad,
                                llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    m.contour(lon,lat,ds_bad_mslp_day_cyclic/100,latlon='true',
                              vmin=970,vmax=1040,levels=np.arange(970,1041,5),cmap='Greys')
                    im = m.contourf(lon,lat,ds_bad_z500_day_cyclic/100,latlon='true',
                              vmin=450,vmax=600,cmap=ncl_colormap(),levels=np.linspace(450,600,30),extend='both')
                    m.contourf(lon,lat,ds_p_z500_day_cyclic,3,latlon='true',
                              colors='none',hatches=[None,'xx'])



                fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0,hspace=0.1)     
                fig.suptitle('Z500 (contours) and MSLP (contour lines) {} {} {} {}'.format(season,event_labelz[j],region,model))

                cbax = fig.add_axes([0.2,0.07,0.6,0.015])

                cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.arange(450,601,25))
                cbar.ax.set_xlabel('Z 500 hPa [m]')
                fig.savefig(path+'fcst_composite/reanalysis/figures/composite_z500_mslp_{}_{}_{}_{}.png'.format(event,season,region,model))
#                 fig.savefig(path+'fcst_composite/reanalysis/figures/try.png'.format(event,season,region,model))
                plt.close(fig)


composite MSLP and Z500 anomaly

In [44]:
plt.rcParams['hatch.color'] = 'k'
seasonz = ['DJF','MAM','JJA','SON','annual']
modelz = ['GFS','ERA5RF']
eventz = ['persistent_hw','persistent_cw']
event_labelz = ['Warm Extreme','Cold Extreme']
file_good_z500 = '/fcst_composite/reanalysis/composite_z500_good_forecasts_{}_{}_{}_{}.nc'
file_bad_z500 = '/fcst_composite/reanalysis/composite_z500_bad_forecasts_{}_{}_{}_{}.nc'
file_p_z500 = '/fcst_composite/reanalysis/composite_z500_diff_p_value_{}_{}_{}_{}.nc'
file_good_z500_anom = '/fcst_composite/reanalysis/composite_z500_anom_good_forecasts_{}_{}_{}_{}.nc'
file_bad_z500_anom = '/fcst_composite/reanalysis/composite_z500_anom_bad_forecasts_{}_{}_{}_{}.nc'
file_p_z500_anom = '/fcst_composite/reanalysis/composite_z500_diff_p_value_{}_{}_{}_{}.nc'

for region in regionz:
    for model in modelz:
        for j, event in enumerate(eventz):
            for season in seasonz:
                fig,axz = plt.subplots(6,2,figsize=(18,12))
                ds_good_z500 = xr.open_dataset(path+file_good_z500.format(event,season,region,model))
                ds_bad_z500 = xr.open_dataset(path+file_bad_z500.format(event,season,region,model))
                ds_good_z500_anom = xr.open_dataset(path+file_good_z500_anom.format(event,season,region,model))
                ds_bad_z500_anom = xr.open_dataset(path+file_bad_z500_anom.format(event,season,region,model))      
                ds_p_z500 = xr.open_dataset(path+file_p_z500.format(event,season,region,model))
                ds_p_z500_anom = xr.open_dataset(path+file_p_z500_anom.format(event,season,region,model))
                days_to_plot = np.arange(-6,5,2)
                for i in range(len(days_to_plot)):
                    ds_p_z500_anom_day = ds_p_z500_anom.sel(days=days_to_plot[i])
                    ds_p_z500_anom_day_bool = ds_p_z500_anom_day.p<0.05
                    ds_p_z500_anom_day_cyclic, lon_cyclic = addcyclic(ds_p_z500_anom_day_bool.values, ds_p_z500_anom_day.longitude.values)
                    ds_good_z500_day = ds_good_z500.sel(time=days_to_plot[i])
                    ds_good_z500_day_cyclic, lon_cyclic = addcyclic(ds_good_z500_day.z.values, ds_good_z500_day.longitude.values)
                    ds_good_z500_anom_day = ds_good_z500_anom.sel(time=days_to_plot[i])
                    ds_good_z500_anom_day_cyclic, lon_cyclic = addcyclic(ds_good_z500_anom_day.z.values, ds_good_z500_anom_day.longitude.values)                
                    lon,lat = np.meshgrid(lon_cyclic,ds_good_z500_day.latitude.values)

                    ax_good = axz[i,0]
                    ax_good.set_title('Good Forecasts Day {}'.format(days_to_plot[i]))
                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_good,
                                llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    m.contour(lon,lat,ds_good_z500_day_cyclic/100,latlon='true',colors='k',
                              vmin=450,vmax=600,levels=np.arange(450,601,15))
                    m.contourf(lon,lat,ds_good_z500_anom_day_cyclic/100,latlon='true',
                              vmin=-25,vmax=25,cmap='bwr',levels=np.linspace(-25,25,51),extend='both')
                    m.contourf(lon,lat,ds_p_z500_anom_day_cyclic,3,latlon='true',
                              colors='none',hatches=[None,'xx'])

                    ds_bad_z500_day = ds_bad_z500.sel(time=days_to_plot[i])
                    ds_bad_z500_day_cyclic, lon_cyclic = addcyclic(ds_bad_z500_day.z.values, ds_bad_z500_day.longitude.values)
                    ds_bad_z500_anom_day = ds_bad_z500_anom.sel(time=days_to_plot[i])
                    ds_bad_z500_anom_day_cyclic, lon_cyclic = addcyclic(ds_bad_z500_anom_day.z.values, ds_bad_z500_anom_day.longitude.values)
                    lon,lat = np.meshgrid(lon_cyclic,ds_bad_z500_day.latitude.values)

                    ax_bad = axz[i,1]
                    ax_bad.set_title('Bad Forecasts Day {}'.format(days_to_plot[i]))

                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_bad,
                                llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    m.contour(lon,lat,ds_bad_z500_day_cyclic/100,latlon='true',colors='k',
                              vmin=450,vmax=600,levels=np.arange(450,601,15))
                    im = m.contourf(lon,lat,ds_bad_z500_anom_day_cyclic/100,latlon='true',
                              vmin=-25,vmax=25,cmap='bwr',levels=np.linspace(-25,25,51),extend='both')
                    m.contourf(lon,lat,ds_p_z500_anom_day_cyclic,3,latlon='true',
                              colors='none',hatches=[None,'xx'])



                fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0,hspace=0.1)     
                fig.suptitle('Z500 Anomaly (contours) and Z500 (contour lines) {} {} {} {}'.format(season,event_labelz[j],region,model))

                cbax = fig.add_axes([0.2,0.07,0.6,0.015])

                cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(-25,25,11))
                cbar.ax.set_xlabel('Z\' 500 hPa [m]')
                fig.savefig(path+'fcst_composite/reanalysis/figures/composite_z500_anom_z500_{}_{}_{}_{}.png'.format(event,season,region,model))
                plt.close(fig)



Composite Daily Precipiation Sum

In [56]:
plt.rcParams['hatch.color'] = 'w'
seasonz = ['DJF','MAM','JJA','SON','annual']
modelz = ['ERA5RF']
eventz = ['persistent_hw','persistent_cw']
event_labelz = ['Warm Extreme','Cold Extreme']
file_good_prec = '/fcst_composite/reanalysis/composite_prec_good_forecasts_{}_{}_{}_{}.nc'
file_bad_prec = '/fcst_composite/reanalysis/composite_prec_bad_forecasts_{}_{}_{}_{}.nc'
file_p_prec = '/fcst_composite/reanalysis/composite_prec_diff_p_value_{}_{}_{}_{}.nc'
clr_levels = [0,0.5,1,2,3,4,5,7.5,10,15,20]

for region in regionz:
    for model in modelz:
        for j, event in enumerate(eventz):
            for season in seasonz:
                fig,axz = plt.subplots(6,2,figsize=(18,14))
                ds_good_prec = xr.open_dataset(path+file_good_prec.format(event,season,region,model))
                ds_bad_prec = xr.open_dataset(path+file_bad_prec.format(event,season,region,model))
                days_to_plot = np.arange(-6,5,2)
                for i in range(len(days_to_plot)):
    #                 ds_p_prec_day = ds_p_prec.sel(days=days_to_plot[i])
    #                 ds_p_prec_day_bool = ds_p_prec_day.p<0.05
    #                 ds_p_prec_day_cyclic, lon_cyclic = addcyclic(ds_p_prec_day_bool.values, ds_p_prec_day.longitude.values)
                    ds_good_prec_day = ds_good_prec.sel(time=days_to_plot[i])
                    ds_good_prec_day_cyclic, lon_cyclic = addcyclic(ds_good_prec_day.tp.values, ds_good_prec_day.longitude.values)                
                    lon,lat = np.meshgrid(lon_cyclic,ds_good_prec_day.latitude.values)

                    ax_good = axz[i,0]
                    ax_good.set_title('Good Forecasts Day {}'.format(days_to_plot[i]))
                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_good,
                                llcrnrlon=-180,llcrnrlat=-15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    m.contourf(lon,lat,ds_good_prec_day_cyclic*1000,latlon='true',
                              vmin=0,vmax=20,cmap='ocean_r',levels=clr_levels,extend='both')
    #                 m.contourf(lon,lat,ds_p_prec_day_cyclic,3,latlon='true',
    #                           colors='none',hatches=[None,'xx'])
                    ds_bad_prec_day = ds_bad_prec.sel(time=days_to_plot[i])
                    ds_bad_prec_day_cyclic, lon_cyclic = addcyclic(ds_bad_prec_day.tp.values, ds_bad_prec_day.longitude.values)
                    lon,lat = np.meshgrid(lon_cyclic,ds_bad_prec_day.latitude.values)

                    ax_bad = axz[i,1]
                    ax_bad.set_title('Bad Forecasts Day {}'.format(days_to_plot[i]))

                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_bad,
                                llcrnrlon=-180,llcrnrlat=-15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    im = m.contourf(lon,lat,ds_bad_prec_day_cyclic*1000,latlon='true',
                              vmin=0,vmax=20,cmap='ocean_r',levels=clr_levels,extend='both')
    #                 m.contourf(lon,lat,ds_p_prec_day_cyclic,3,latlon='true',
    #                           colors='none',hatches=[None,'xx'])



                fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0,hspace=0.1)     
                fig.suptitle('Daily Precipitation Sum {} {} {} {}'.format(season,event_labelz[j],region,model))

                cbax = fig.add_axes([0.2,0.07,0.6,0.015])

                cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=clr_levels)
                cbar.ax.set_xlabel('P_sum [mm]')
                sys.exit()
                fig.savefig(path+'fcst_composite/reanalysis/figures/composite_prec_{}_{}_{}_{}.png'.format(event,season,region,model))
                plt.close(fig)



SystemExit: 

Composite RWP Envelope

In [41]:
plt.rcParams['hatch.color'] = 'w'
seasonz = ['annual']
modelz = ['GFS','ERA5RF']
eventz = ['persistent_hw','persistent_cw']
event_labelz = ['Warm Extreme','Cold Extreme']
file_good_env = '/fcst_composite/reanalysis/composite_env_good_forecasts_{}_{}_{}_{}.nc'
file_bad_env = '/fcst_composite/reanalysis/composite_env_bad_forecasts_{}_{}_{}_{}.nc'
file_p_env = '/fcst_composite/reanalysis/composite_env_diff_p_value_{}_{}_{}_{}.nc'

for region in regionz:
    for model in modelz:
        for j, event in enumerate(eventz):
            for season in seasonz:
                fig,axz = plt.subplots(6,2,figsize=(18,12))
                ds_good_env = xr.open_dataset(path+file_good_env.format(event,season,region,model))
                ds_bad_env = xr.open_dataset(path+file_bad_env.format(event,season,region,model))
                ds_p_env = xr.open_dataset(path+file_p_env.format(event,season,region,model))

                days_to_plot = np.arange(-6,5,2)
                for i in range(len(days_to_plot)):
                    ds_p_env_day = ds_p_env.sel(days=days_to_plot[i])
                    ds_p_env_day_bool = ds_p_env_day.p<0.05
                    ds_p_env_day_cyclic, lon_cyclic = addcyclic(ds_p_env_day_bool.values, ds_p_env_day.longitude.values)
                    ds_good_env_day = ds_good_env.sel(time=days_to_plot[i])
                    ds_good_env_day_cyclic, lon_cyclic = addcyclic(ds_good_env_day.v.values, ds_good_env_day.longitude.values)                
                    lon,lat = np.meshgrid(lon_cyclic,ds_good_env_day.latitude.values)

                    ax_good = axz[i,0]
                    ax_good.set_title('Good Forecasts Day {}'.format(days_to_plot[i]))
                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_good,
                                llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    m.contourf(lon,lat,ds_good_env_day_cyclic,latlon='true',
                              vmin=0,vmax=50,cmap=ncl_colormap(),levels=np.linspace(0,50,26),extend='both')
                    m.contourf(lon,lat,ds_p_env_day_cyclic,3,latlon='true',
                              colors='none',hatches=[None,'xx'])

                    ds_bad_env_day = ds_bad_env.sel(time=days_to_plot[i])
                    ds_bad_env_day_cyclic, lon_cyclic = addcyclic(ds_bad_env_day.v.values, ds_bad_env_day.longitude.values)
                    lon,lat = np.meshgrid(lon_cyclic,ds_bad_env_day.latitude.values)

                    ax_bad = axz[i,1]
                    ax_bad.set_title('Bad Forecasts Day {}'.format(days_to_plot[i]))

                    m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_bad,
                                llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                    m.drawcoastlines()
                    im = m.contourf(lon,lat,ds_bad_env_day_cyclic,latlon='true',
                              vmin=0,vmax=50,cmap=ncl_colormap(),levels=np.linspace(0,50,26),extend='both')
                    m.contourf(lon,lat,ds_p_env_day_cyclic,3,latlon='true',
                              colors='none',hatches=[None,'xx'])



                fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0,hspace=0.1)     
                fig.suptitle('RWP Envelope {} {} {} {}'.format(season,event_labelz[j],region,model))

                cbax = fig.add_axes([0.2,0.07,0.6,0.015])

                cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.arange(0,51,10))
                cbar.ax.set_xlabel('E [/sm]')
                fig.savefig(path+'fcst_composite/reanalysis/figures/composite_env_{}_{}_{}_{}.png'.format(event,season,region,model))
                plt.close(fig)




In [ ]:
plt.rcParams['hatch.color'] = 'k'
seasonz = ['DJF','MAM','JJA','SON']
modelz = ['GFS','ERA5RF']
eventz = ['persistent_hw','persistent_cw']
event_labelz = ['Warm Extreme','Cold Extreme']
file_good_cp = '/fcst_composite/reanalysis/composite_cp_good_forecasts_{}_{}_{}_{}.nc'
file_bad_cp = '/fcst_composite/reanalysis/composite_cp_bad_forecasts_{}_{}_{}_{}.nc'
file_p_cp = '/fcst_composite/reanalysis/composite_cp_diff_p_value_{}_{}_{}_{}.nc'

for model in modelz:
    for j, event in enumerate(eventz):
        for season in seasonz:
            fig,axz = plt.subplots(3,4,figsize=(20,9))
            ds_good_cp = xr.open_dataset(path+file_good_cp.format(event,season,region,model))
            ds_bad_cp = xr.open_dataset(path+file_bad_cp.format(event,season,region,model))
            ds_p_cp = xr.open_dataset(path+file_p_cp.format(event,season,region,model))

            days_to_plot = np.arange(-6,5,2)
            for i in range(len(days_to_plot)):
                ds_p_cp_day = ds_p_cp.sel(days=days_to_plot[i])
                ds_p_cp_day_bool = ds_p_cp_day.p<0.05
                ds_p_cp_day_cyclic, lon_cyclic = addcyclic(ds_p_cp_day_bool.values, ds_p_cp_day.longitude.values)
                ds_good_cp_day = ds_good_cp.sel(time=days_to_plot[i])
                ds_good_cp_day_cyclic, lon_cyclic = addcyclic(ds_good_cp_day.v.values, ds_good_cp_day.longitude.values)                
                lon,lat = np.meshgrid(lon_cyclic,ds_good_cp_day.latitude.values)
                
                ax_good = axz[i%3,(i//3)*2]
                ax_good.set_title('Good Forecasts Day {}'.format(days_to_plot[i]))
                m = Basemap(projection='merc',lat_0=50,lon_0=-40,resolution='c',ax=ax_good,
                    llcrnrlon=-150,llcrnrlat=15,urcrnrlon=60,urcrnrlat=75)
                m.drawcoastlines()
                m.contourf(lon,lat,ds_good_cp_day_cyclic,latlon='true',alpha=.5,antialiased=True,
                          vmin=-10,vmax=10,cmap='bwr',levels=np.linspace(-5,10,16),extend='both')
                m.contourf(lon,lat,ds_p_cp_day_cyclic,3,latlon='true',
                          colors='none',hatches=[None,'xx'])
                
                ds_bad_cp_day = ds_bad_cp.sel(time=days_to_plot[i])
                ds_bad_cp_day_cyclic, lon_cyclic = addcyclic(ds_bad_cp_day.v.values, ds_bad_cp_day.longitude.values)
                lon,lat = np.meshgrid(lon_cyclic,ds_bad_cp_day.latitude.values)
                
                ax_bad = axz[i%3,(i//3)*2+1]
                ax_bad.set_title('Bad Forecasts Day {}'.format(days_to_plot[i]))

                m = Basemap(projection='merc',lat_0=50,lon_0=-40,resolution='c',ax=ax_bad,
                    llcrnrlon=-150,llcrnrlat=15,urcrnrlon=60,urcrnrlat=75)
                m.drawcoastlines()
                im = m.contourf(lon,lat,ds_bad_cp_day_cyclic,latlon='true',alpha=.5,antialiased=True,
                          vmin=-10,vmax=10,cmap='bwr',levels=np.linspace(-5,10,16),extend='both')
                m.contourf(lon,lat,ds_p_cp_day_cyclic,3,latlon='true',
                          colors='none',hatches=[None,'xx'])
                

                
            fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0,hspace=0.1)     
            fig.suptitle('RWP Phase Speed {} {} {}'.format(season,event_labelz[j],model))
            
            cbax = fig.add_axes([0.2,0.07,0.6,0.015])

            cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(-5,10,4))
            cbar.ax.set_xlabel('Cp [m/s]')
            fig.savefig(path+'fcst_composite/reanalysis/figures/composite_cp_{}_{}_{}_{}.png'.format(event,season,region,model))
            plt.close(fig)
            

